In [32]:
!pip install streamlit ngrok

In [33]:
# Install pyngrok to use ngrok within Colab
!pip install pyngrok


In [34]:
!pip install transformers

In [35]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_path = "ayaht/flant5finetuneddepi"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the functions to generate question and answer
def generate_question(context):
    input_text = f"Generate a question for this context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_answer(context, question):
    input_text = f"Answer this question based on the given context. Question: {question} Context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        output = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def generate_qa(context):
    question = generate_question(context)
    answer = generate_answer(context, question)
    return question, answer

# Step 3: Create the Streamlit app interface
st.title("Q-Ease")
st.write("We developed a Question Answering (QA) model to help students with challenging subjects like biology and chemistry.The model generates questions and provides accurate answers to improve understanding of scientific material.")

# Input text box for context
context = st.text_area("Enter context here:", height=200)

# Button to generate question and answer
if st.button("Generate Question and Answer"):
    if context:
        with st.spinner('Generating...'):
            generated_question, generated_answer = generate_qa(context)
            st.write(f"*Generated Question:* {generated_question}")
            st.write(f"*Generated Answer:* {generated_answer}")
    else:
        st.warning("Please provide a context before generating.")

Overwriting app.py


In [37]:
import ngrok

# Authenticate ngrok
!ngrok authtoken 2nbneWvK7u8G10PfC1fEYhzRly2_3nhX4ByePFJmormajwD7R

# Create a tunnel on port 8501 (the default Streamlit port)
public_url = ngrok.connect(port='8501')
public_url

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


<Task pending name='Task-8' coro=<wrap() running at ngrok_wrapper:6>>

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com


34.91.117.194
your url is: https://cold-otters-give.loca.lt
